In [2]:
import pandas as pd
import numpy as np

In [3]:
winds_df = pd.read_csv('../datasets/tiny-imagenet-200/wnids.txt', sep=" ", header=None)
winds_df.columns = ["winds"]

## Getting the names for all classes

In [4]:
words_winds_df = pd.read_csv('../datasets/tiny-imagenet-200/words.txt', sep="\t", header=None)
words_winds_df.columns = ["winds", "classes"]

In [6]:
tiny_imagenet_classes = pd.merge(words_winds_df, winds_df, how='inner', on=['winds'])

In [7]:
tiny_imagenet_classes.head()

,winds,classes
0,n01443537,"goldfish, Carassius auratus"
1,n01629819,"European fire salamander, Salamandra salamandra"
2,n01641577,"bullfrog, Rana catesbeiana"
3,n01644900,"tailed frog, bell toad, ribbed toad, tailed to..."
4,n01698640,"American alligator, Alligator mississipiensis"


In [9]:
tiny_imagenet_classes.to_csv('../datasets/tiny-imagenet-200/class_labels.csv', index=False)

## Getting the hierarchy of all classes 

In [76]:
import nltk
from nltk.corpus import wordnet

def get_parent(wnid):
    parents = None
    parents = wnid.hypernyms()
    if len(parents)==0:
        return None
#     elif len(parents)>1:
#         print('many to one mapping: {}'.format(str(parents)))
    else:
        return parents[0]
    
def get_all_parents(wnid):
    hierachy_list = []
    parent_wnid = get_parent(wnid)
    while parent_wnid:
        hierachy_list.append(parent_wnid)
        parent_wnid = get_parent(parent_wnid)
    
    return hierachy_list

In [77]:
hierarchy_list = []
for index, row in tiny_imagenet_classes.iterrows():
    wnid = row['winds']
    wnid = int(wnid.strip('n'))
    hierarchy = get_all_parents(wn.synset_from_pos_and_offset('n',wnid))
    level_list = []
    for level in hierarchy:
        level_list.append(level.lemma_names())
    hierarchy_list.append(level_list)

In [87]:
hierarchy_df = pd.DataFrame(hierarchy_list)
hierarchy_df['winds'] = tiny_imagenet_classes['winds']
tiny_imagenet_with_hierarchy = pd.merge(tiny_imagenet_classes,hierarchy_df)
tiny_imagenet_with_hierarchy.head(2)

,winds,classes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,n01443537,"goldfish, Carassius auratus","[cyprinid, cyprinid_fish]",[cypriniform_fish],"[soft-finned_fish, malacopterygian]","[teleost_fish, teleost, teleostan]",[bony_fish],[fish],[aquatic_vertebrate],"[vertebrate, craniate]",[chordate],"[animal, animate_being, beast, brute, creature...","[organism, being]","[living_thing, animate_thing]","[whole, unit]","[object, physical_object]",[physical_entity],[entity],None
1,n01629819,"European fire salamander, Salamandra salamandra",[salamander],[amphibian],"[vertebrate, craniate]",[chordate],"[animal, animate_being, beast, brute, creature...","[organism, being]","[living_thing, animate_thing]","[whole, unit]","[object, physical_object]",[physical_entity],[entity],None,None,None,None,None,None


In [88]:
tiny_imagenet_with_hierarchy.to_csv('../datasets/tiny-imagenet-200/tiny_imagenet_with_hierarchy.csv', index=False)